In [1]:
import sklearn
import re
import nltk
import csv
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer

import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

global Y, vector

df = pd.read_csv(r'C:\Users\melan\OneDrive\Documents\drugsTrain2.csv', 
                 names = ['Name', 'Condition', 'Review', 'Rating'])
#(r'C:\Users\melan\OneDrive\Documents\drugsTrain2.csv', skiprow = skip)
'''filtered_rows = []
Ratings = [1, 10]

for row in df:
    if row['Rating'] in Ratings:
        filtered_rows.append(row)'''
df = df.drop(['Name', 'Condition'], 1)

#select the ratings with 1 and 10
high = df.loc[df['Rating'] == 10]
low = df.loc[df['Rating'] == 1]

#combine the dataframes
frames = [low, high]
drugs_df = pd.concat(frames)

Y = drugs_df['Rating']
'''#df = next(df)
df.to_csv('newdrugs1.csv')
df.head(10)'''

"#df = next(df)\ndf.to_csv('newdrugs1.csv')\ndf.head(10)"

#make the corpus lowercase
reviews = drugs_df['Review'].tolist()
for i in range(len(reviews)):
    reviews[i] = reviews[i].lower()

"""#remove stopwords
join = " ".join(reviews)
preview = word_tokenize(join)
#text = [word for word in preview if word not in set(stopwords.words('english'))]
filtered = []
stop = stopwords.words("english")
for word in preview:
    if word not in stop:
        filtered.append(word)
join2 = " ".join(filtered)
token = word_tokenize(join2)
print(filtered[:4])
print(token[:4])

   
for word in reviews:
    r = word.split()
    if word not in stop:
        text1.append(word)
 """       
vectorizer = TfidfVectorizer()
temp = reviews[:4]
ftm = vectorizer.fit_transform(temp)
tfidf = TfidfTransformer()
tf_trans = tfidf.fit_transform(ftm)
final = pd.DataFrame(tf_trans.toarray(), columns=vectorizer.get_feature_names())
print(tf_trans.toarray())
#matrix


In [2]:
reviews = drugs_df['Review']
temp = reviews
stop_words = set(stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
vector = tfidf_vectorizer.fit_transform(temp)
#l = vector.toarray()
vectorizer = tfidf_vectorizer.transform(temp)

classifier = MultinomialNB()
x_train, x_test, y_train, y_test = train_test_split(vectorizer, Y, test_size = 0.4)
classify = MultinomialNB().fit(x_train, y_train)
y_pred = classify.predict(x_test)
score = accuracy_score(y_test, y_pred)
#classification_report = metrics.classification_report(y_test, y_pred)
print(score)
print(classification_report(y_test, y_pred))
cmat = confusion_matrix(y_test, y_pred)
print(cmat)

#visualizing the confusion matrix
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(cmat)
ax.grid(False)
ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 10s', 'Predicted 1s'))
ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 10s', 'Actual 1s'))
ax.set_ylim(1.5, -0.5)
sns.heatmap(pd.DataFrame(cmat), annot=True, cmap="YlGnBu" ,fmt='g')
for i in range(2):
    for k in range(2):
        ax.text(k, i, cmat[i, k], ha='center', va='center', color='white')
plt.show()

#top words in both datasets
toplow_sorted = classify.feature_log_prob_[0, :].argsort()[::-1]
tophigh_sorted = classify.feature_log_prob_[1, :].argsort()[::-1]

print(np.take(tfidf_vectorizer.get_feature_names_out(), toplow_sorted[:10]))
print(np.take(tfidf_vectorizer.get_feature_names_out(), tophigh_sorted[:10]))

0.7815039250791902
              precision    recall  f1-score   support

           1       0.95      0.27      0.43      8590
          10       0.77      0.99      0.87     20454

    accuracy                           0.78     29044
   macro avg       0.86      0.63      0.65     29044
weighted avg       0.82      0.78      0.74     29044

[[ 2361  6229]
 [  117 20337]]


NameError: name 'plt' is not defined

In [ ]:
#Logistic Regression
model = LogisticRegression(C=1, class_weight=None, max_iter=1000)
x_tr, x_t, y_tr, y_t = train_test_split(vector, Y, test_size = 0.4)
fitting = model.fit(x_tr, y_tr)
testing = fitting.score(x_t, y_t)
pred = model.predict(x_t)

#cross validation 10 folds
kfold = KFold(n_splits=10, random_state=0, shuffle=True)
scores = cross_val_score(fitting, vector, Y, scoring='accuracy', cv=kfold)

#confusion matrix
cmat_lr = confusion_matrix(y_t, pred)
figure, m = plt.subplots(figsize=(8, 8))
m.imshow(cmat_lr)
m.grid(False)
m.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 10s', 'Predicted 1s'))
m.yaxis.set(ticks=(0, 1), ticklabels=('Actual 10s', 'Actual 1s'))
m.set_ylim(1.5, -0.5)
sns.heatmap(pd.DataFrame(cmat_lr), annot=True, cmap="YlGnBu" ,fmt='g')
for o in range(2):
    for p in range(2):
        m.text(o, p, cmat[o, p], ha='center', va='center', color='white')
plt.show()
print("Accuracy: %.3f%%, standard dev: %.3f%%" % (scores.mean()*100.0, scores.std()*100.0))
print("Raw accuracy score:", testing)
print("************Classification Report Regression Model************\n", classification_report(y_t, pred))

#ROC curve
y_pred_proba = model.predict(x_t)
roc, curve, l = metrics.roc_curve(y_t,  y_pred_proba, pos_label = 10)
rocc = metrics.roc_auc_score(y_t, y_pred_proba)
plt.plot(roc, curve, label="drugs data, auc=" + str(rocc))
plt.legend(loc=4)
plt.show()
#cm_reg = confusion_matrix(x_t, model.predict(y_t))
print(scores)

In [ ]:
fl = pd.read_csv("tf_idf.csv")
frame = pd.DataFrame(fl)
#frame.head(10)
frame.shape

In [ ]:
import sys
sys.setrecursionlimit(20000)
from sklearn.metrics import accuracy_score, classification_report
'''def classification(y_test, y_pred):
    
    classification(y_test, y_pred)
    score = accuracy_score(y_test, y_pred, normalize=True)
    
    print("accuracy_score : ", score)
'''
X = frame.iloc[:, 0]
#x = np.asarray(X)
#y = np.asarray(Y)
classifier = MultinomialNB()
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.4)
classifier.fit(x_train, y_train)
classify = GaussianNB().fit(x_train, y_train)
y_pred = classify.predict(x_test)   
#classification(y_test, y_pred)
#score = accuracy_score(y_test, y_pred, normalize=True)
classification_report = classification_report(y_test, y_pred)
print("accuracy_score : ", y_test, " ", y_pred)

In [ ]:


'''file = open('tf_idf.csv', 'w+', newline ='') 
with file:     
    write = csv.writer(file) 
    write.writerows(l)'''

"""
default=None
(500, 3948)
(500, 4191)

countvec = CountVectorizer(stop_words='english')
vectorizer = TfidfTransformer()
ftm = countvec.fit_transform(temp)
tfidf = vectorizer.fit_transform(ftm)
r = tfidf.toarray()
#print(temp)
print(r[:1])

ftm = vectorizer.fit_transform(temp)
tfidf = TfidfTransformer(norm='12')
#tf_trans = tfidf.fit_transform(ftm)"""
#terms = vectorizer.get_feature_names()
#df = pd.DataFrame(r, id_vars=["Rating"])
#final = pd.DataFrame(r, columns= ['Review', 'Rating'])
"""
final = pd.DataFrame(r, columns= ['Review', 'Rating'])
ratings = pd.DataFrame(drugs_df['rating'])
da = pd.DataFrame(final, columns = ['Ratings', 'Review'])
frame = pd.concat([ratings, final])
#new = pd.crosstab(frame.ratings, df.final)
k = pd.DataFrame(r)
k = pd.melt(k)

for i in r:
    for k in i:
        print(k,end = " ")

res = " ".join([str(i) for i in r])
file = open('tf_idf.csv', 'w+', newline ='') 
with file:     
    write = csv.writer(file) 
    write.writerows(r) 
"""
'''
array = [[9 834], [74 0.0], [125 485 576]] 
file = open('test.csv', 'w+', newline ='') 
with file:     
    write = csv.writer(file) 
    write.writerows(array) 


cvectorizer = CountVectorizer()
_self_.cvectorizer = cvectorizer
term = _self_.cvectorizer.transform(temp)
vectorizer.fit(term)
k = vectorizer.transform(term)

tfidf = TfidfTransformer(norm='l2')
term_vectors.todense()
#[0, 1, 1, 1]
# [0, 1, 0, 2]
tfidf.fit(term_vectors)
tf_idf_matrix = tfidf.transform(term_vectors)
print("\nVector of idf \n")
print(tfidf.idf_)
print("\nFinal tf-idf vectorizer matrix form :\n")
print(tf_idf_matrix.todense())
newlist = []
for lines in reviews:
    lines = lines.split("\n")
    newlist.append(lines)



reviews = drugs_df['review'].tolist()
print(reviews[:4])
#reviews = reviews.values.tolist()
#ftm = vectorizer.fit_transform(reviews)
newlist = []
for lines in reviews:
    lines = lines.split("\n")
    newlist.append(lines)
#print(newlist[:4])
#ftm = vectorizer.fit_transform(newlist)
tfidf = TfidfTransformer()
tf_trans = tfidf.fit_transform(ftm)
vectorizer.idf_'''

"""
drugs_df = pd.read_csv(r'C:\Users\melan\OneDrive\Documents\drugsTrain2.csv', 
                       skiprows = 1, names = ['Drug', 'Condition', 'Review', 'Rating'])

#check to see if any null or empty fields
drugs_df[drugs_df.isnull().any(axis=1)].count()
drugs_df = drugs_df.dropna()

drugs_corr = drugs_df.corr()"""